In [ ]:
import os
import glob
import numpy as np
from numpy import mean, std
import matplotlib.pyplot as plt
import tensorflow as tf
from pandas import read_csv
import keras.backend as kb
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Run for GPU/CUDA ML, comment out if you dont have it configured.
window = 20 # the length of time series data
gpu_devices = tf.config.list_physical_devices('GPU')

In [ ]:
directory = "/content/drive/MyDrive/CS3237 Group Project/Gesture Recognition Model/"
gestures = ["gestureA", "gestureB"]

NUM_FEATURES = 6
TIMESTEP = 20
NUM_GESTURES = len(gestures)

x_train_list = []
y_train_list = []

for gesture in gestures:
    all_data = []
    gesture_dir = os.path.join(directory, gesture)
    all_gesture_files = glob.glob(gesture_dir + "/**/*.csv", recursive=True)

    num_files = len(all_gesture_files)

    for file in all_gesture_files:
        filename = os.path.join(gesture_dir, file)
        data = np.loadtxt(filename, delimiter=',', skiprows=1)
        all_data.append(data)

    final_data = np.concatenate(all_data, axis=0 )
    final_data = np.reshape(final_data, (num_files, TIMESTEP, NUM_FEATURES))

    labels = np.zeros((num_files, NUM_GESTURES))
    lbl_idx = gestures.index(gesture)
    labels[:, lbl_idx] = 1

    x_train_list.append(final_data)
    y_train_list.append(labels)

x_train = np.concatenate(x_train_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)

print(f"X_train shape: {x_train.shape}")
print(f"Y_train shape: {y_train.shape}")

X_train shape: (2053, 20, 6)
Y_train shape: (2053, 2)


In [ ]:
def visualise(history, label):
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])

    plt.title('Accuracy of LSTM Model')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(f'/content/drive/MyDrive/CS3237 Group Project/Gesture Recognition Model/{label}')
    plt.close()


In [ ]:
def evaluate_lstm(x_train, y_train, dropout, LR=0.0001, epochs=20, units=100, label='fig'):
    print("start evaluation!")
    verbose, batch_size = 1, 64
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    ##LSTM model structure
    model = Sequential()
    model.add(LSTM(units, input_shape=(n_timesteps, n_features), dropout=dropout))
    model.add(Dropout(dropout))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    opt = Adam(lr=LR)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy()])

    model.summary()
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                        validation_split=0.15,  shuffle=True)
    ##plot
    visualise(history, label)

    # save model
    if not os.path.exists('lstm_models'):
        os.makedirs('lstm_models')
    model_name = os.path.join(directory, "model")
    model.save(model_name)


In [ ]:
# Run dropout experiment
dropouts = [0.0, 0.2, 0.4]
lr = 0.0001
unit = 100

for dropout in dropouts:
    label = f'fig_dropout{dropout}_lr{lr}_units{unit}'
    label = label.replace('.', ',')
    score = evaluate_lstm(x_train, y_train, dropout, LR=lr, units=unit, label=label)


In [ ]:
# Run lr experiment
dropout = 0.4
learning_rates = [0.1, 0.01, 0.001, 0.0001]
unit = 100

for lr in learning_rates:
    label = f'fig_dropout{dropout}_lr{lr}_units{unit}'
    label = label.replace('.', ',')
    score = evaluate_lstm(x_train, y_train, dropout, LR=lr, units=unit, label=label)


In [ ]:
# Run units experiment
dropout = 0.4
lr = 0.0001
units = [5, 10, 20, 50, 100]

for unit in units:
    label = f'fig_dropout{dropout}_lr{lr}_units{unit}'
    label = label.replace('.', ',')
    score = evaluate_lstm(x_train, y_train, dropout, LR=lr, units=unit, label=label)